# Baseline table - Sepsis manuscript

This notebook generates the baseline table for the sepsis manuscript.

First, lets setup the notebook settings:

In [1]:
%cd ../src
%pwd
%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\admin\Desktop\Sepsis\src


Next, we import all important libaries.

In [8]:
# general imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import string
import datetime as dt
import numpy as np

# utils import
from utils.files import get_latest_version
from utils.cross_validation import cross_validate_ROC, cross_validate_risk_ROC, compare_models
from utils.risk_scores import read_data_risk_score

# xgboost
from xgboost import XGBClassifier

First, we read all the data from the dataframe

In [3]:
model_version = 24
model_dir = [x for x in os.listdir(os.path.join(os.getcwd(), '..', 
                                                    'models')) if str(model_version).zfill(3) in x][0]

model_main_file = "model_{}_out_{}_maindb.csv".format(3, 1)
model_val_file = "model_{}_out_{}_validationdb.csv".format(3, 1)

df = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_main_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_valid = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_val_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_total = pd.concat([df, df_valid])

Now we first define the _abbMEDS function according to the paper of
Vorwerk et al 2009.

In [4]:
# meds clinical risk score
def _abbMEDS(row):
    points = 0
    
    #rule 1
    if row['metastase_of_chronische_ziekte_met_hoge_mortaliteit'] == 1:
        points += 6
        
    #rule 2
    if row['AF_berekend'] > 20:
        points += 3
    
    #rule 3
    if row['y_var'] == 1: # hardcode septic shock
        points += 3
    
    #rule 4
    if row['TRC'] < 150:
        points += 3
    
    #rule 5
    if row['Geboortedatum'] >= 65:
        points += 3
        
    #rule 6        
    if row['WD'] == 2:
        points += 2
    
    #rule 7
    if row['Woonsituatie'] in [2,3]:
        points += 2
        
    #rule 8   
    if row['Comorb_dementie_Psychiatrisch']:
        points += 2
        
    return points

Next we define the mREMS clinical risk score according to Crowe et al. 2010

In [5]:
def _mREMS(row):
    points = 0
    
    #rule 1
    points += np.where(row['Geboortedatum'] < 45, 0, 
                       np.where(row['Geboortedatum'] < 54, 2,
                        np.where(row['Geboortedatum'] < 64, 3, 
                         np.where(row['Geboortedatum'] < 74, 5, 6))))
    # rule 2
    points += np.where(row['Pols'] < 39, 4,
               np.where(row['Pols'] < 55, 3,
                 np.where(row['Pols'] < 70, 2,
                  np.where(row['Pols'] < 110, 0,
                   np.where(row['Pols'] < 139, 2,
                    np.where(row['Pols'] < 179, 3, 4))))))
    
    # rule 3
    points += np.where(row['AF_berekend'] < 5, 4,
               np.where(row['AF_berekend'] < 9, 3,
                 np.where(row['AF_berekend'] < 12, 1,
                  np.where(row['AF_berekend'] < 25, 0,
                   np.where(row['AF_berekend'] < 35, 1,
                    np.where(row['AF_berekend'] < 50, 3, 4))))))
    
    # rule 4
    RR_map = ((row['RR_syst'] + 2 * row['RR_diast'])/3)
    points += np.where(RR_map < 49, 4,
               np.where(RR_map < 70, 2,
                np.where(RR_map < 110, 0,
                 np.where(RR_map < 130, 2,
                  np.where(RR_map < 160, 3, 4)))))
    
    # rule 5
    points += np.where(row['Saturatie'] == 0, 0,
                np.where(row['Saturatie'] < 75, 4,
                 np.where(row['Saturatie'] < 85, 3,
                  np.where(row['Saturatie'] < 90, 1, 0))))
    
    #rule 6
    points += np.where(row['GCS_afgeleid'] > 13, 1,
                np.where(row['GCS_afgeleid'] > 11, 1,
                 np.where(row['GCS_afgeleid'] > 8, 2,
                  np.where(row['GCS_afgeleid'] > 5, 3, 4))))
    
    if row['Comorb_dementie_Psychiatrisch']:
        points += 1
        
    return points

Define a small helper function

In [6]:
def _score_to_risk(_df):
    df['abbMEDS_risk'] = np.where(df['abbMEDS'] < 5, 0.016, 
                                  np.where(df['abbMEDS'] < 13, 0.234, 0.59))
    return _df

Next, we calculate the scores for each of the dataframes:
    - df ; main DB
    - df_valid ; validation DB
    - df_total ; total DB

In [11]:
df['abbMEDS'] = df.apply(_abbMEDS, axis=1)
df['mREMS'] = df.apply(_mREMS, axis=1)

df_valid['abbMEDS'] = df_valid.apply(_abbMEDS, axis=1)
df_valid['mREMS'] = df_valid.apply(_mREMS, axis=1)

df_total['abbMEDS'] = df_total.apply(_abbMEDS, axis=1)
df_total['mREMS'] = df_total.apply(_mREMS, axis=1)

print(list(df_valid['mREMS']))

[4, 9, 6, 10, 4, 6, 10, 3, 7, 1, 9, 7, 1, 6, 11, 11, 6, 7, 9, 3, 7, 8, 3, 7, 1, 6, 6, 8, 12, 11, 7, 13, 7, 7, 5, 9, 8, 5, 12, 1, 9, 10, 7, 4, 7, 7, 10, 11, 9, 4, 6, 5, 9, 10, 7, 7, 4, 8, 7, 10, 12, 3, 8, 6, 6, 7, 1, 9, 9, 9, 6, 8, 6, 8, 4, 5, 7, 10, 7, 7, 11, 7, 6, 7, 6, 1, 7, 10, 10, 7, 11, 1, 3, 7, 10, 8, 7, 7, 3, 11]


Calculate median +- IQR for each of the scores

In [173]:
for d in [df, df_valid, df_total]:
    print(d['abbMEDS'].describe())
    print(d['mREMS'].describe())

count    1244.000000
mean        5.761254
std         3.958767
min         0.000000
25%         3.000000
50%         6.000000
75%         8.000000
max        20.000000
Name: abbMEDS, dtype: float64
count    1244.000000
mean        7.357717
std         2.967756
min         1.000000
25%         6.000000
50%         7.000000
75%         9.000000
max        19.000000
Name: mREMS, dtype: float64
count    100.000000
mean       5.750000
std        3.748063
min        0.000000
25%        3.000000
50%        5.500000
75%        8.000000
max       17.000000
Name: abbMEDS, dtype: float64
count    100.000000
mean       7.020000
std        2.828356
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       13.000000
Name: mREMS, dtype: float64
count    1344.000000
mean        5.760417
std         3.942141
min         0.000000
25%         3.000000
50%         6.000000
75%         8.000000
max        20.000000
Name: abbMEDS, dtype: float64
count    1344.000000
mean     

We should use mann-whitney U to check for differences between the two groups

In [175]:
from scipy.stats import mannwhitneyu
print(mannwhitneyu(df['abbMEDS'], df_valid['abbMEDS']))
print(mannwhitneyu(df['mREMS'], df_valid['mREMS']))

MannwhitneyuResult(statistic=62056.0, pvalue=0.4845207181967723)
MannwhitneyuResult(statistic=59154.5, pvalue=0.2055895745049367)


Next, we read all data for output 1 (shock) and output (3) mortality into dataframes

In [163]:
model_version = 24
model_dir = [x for x in os.listdir(os.path.join(os.getcwd(), '..', 
                                                    'models')) if str(model_version).zfill(3) in x][0]

model_main_file = "model_{}_out_{}_maindb.csv".format(3, 1)
model_val_file = "model_{}_out_{}_validationdb.csv".format(3, 1)

df_main_shock = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_main_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_valid_shock = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_val_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_total_shock = pd.concat([df, df_valid])

In [178]:
model_version = 22
model_dir = [x for x in os.listdir(os.path.join(os.getcwd(), '..', 
                                                    'models')) if str(model_version).zfill(3) in x][0]

model_main_file = "model_{}_out_{}_maindb.csv".format(3, 3)
model_val_file = "model_{}_out_{}_validationdb.csv".format(3, 3)

df_main_mort = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_main_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_valid_mort = pd.read_csv(os.path.join(os.getcwd(), '..', 'models', model_dir, model_val_file), 
                      sep =',', 
                      header=0,
                      infer_datetime_format=True, 
                      error_bad_lines=False, 
                      engine='python',
                      encoding='utf-8')

df_total_mort = pd.concat([df_main_mort, df_valid_mort])

Calculate the relative frequencies of both

In [179]:
for d in [df_main_shock, df_valid_shock, df_total_shock]:
    print(d['y_var'].value_counts())
    
for d in [df_main_mort, df_valid_mort, df_total_mort]:
    print(d['y_var'].value_counts())

0    1150
1      94
Name: y_var, dtype: int64
0    92
1     8
Name: y_var, dtype: int64
0    1242
1     102
Name: y_var, dtype: int64
0    1083
1     161
Name: y_var, dtype: int64
0    87
1    13
Name: y_var, dtype: int64
0    1170
1     174
Name: y_var, dtype: int64


Define our own chi_square function

In [177]:
def chi_square(maincol, validcol):
    values = list(maincol) + list(validcol)
    _df = pd.DataFrame({'values' : values, 'cohort' : [0] * len(maincol) + [1] * len(validcol)})
    return chi2_contingency(pd.crosstab(_df['values'], _df['cohort']))

Calculate chi-square values for both septic shock as well as mortality

In [170]:
chi_square(df_main_shock['y_var'], df_valid_shock['y_var'])

(0.0012280600762603275,
 0.9720449041777297,
 1,
 array([[1149.58928571,   92.41071429],
        [  94.41071429,    7.58928571]]))

In [171]:
chi_square(df_main_mort['y_var'], df_valid_mort['y_var'])

(0.019104966438373797,
 0.890065982933238,
 1,
 array([[1082.94642857,   87.05357143],
        [ 161.05357143,   12.94642857]]))